In [1]:
# Update sklearn to prevent version mismatches
!pip install sklearn --upgrade

In [2]:
# install joblib. This will be used to save your model. 
# Restart your kernel after installing 
!pip install joblib

In [3]:
import pandas as pd
import tensorflow
tensorflow.keras.__version__

'2.4.0'

# Read the CSV and Perform Basic Data Cleaning

In [4]:
df = pd.read_csv("exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


In [ ]:
df.columns

# Select your features (columns)

In [5]:
# Set features. This will also be used as your x values.
# check the definition of each column name from:  https://exoplanetarchive.ipac.caltech.edu/docs/API_kepcandidate_columns.html#pdisposition
selected_features = df[['koi_fpflag_nt', 'koi_fpflag_ss', 'koi_fpflag_co',
       'koi_fpflag_ec', 'koi_period', 
        'koi_impact',
        'koi_duration',
        'koi_depth', 
        'koi_prad', 
       'koi_teq', 'koi_insol', 
       'koi_model_snr',  'koi_steff', 
        'koi_slogg', 
       'koi_srad',  
       'koi_kepmag']]

# Create a Train Test Split

Use `koi_disposition` for the y values

In [6]:
target = df["koi_disposition" ]


In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(selected_features, target, random_state=42)

In [8]:
X_train.head()

,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_impact,koi_duration,koi_depth,koi_prad,koi_teq,koi_insol,koi_model_snr,koi_steff,koi_slogg,koi_srad,koi_kepmag
6122,0,0,0,0,6.768901,0.150,3.61600,123.1,1.24,1017,253.30,10.8,5737,4.327,1.125,14.725
6370,0,1,0,1,0.733726,0.291,2.30900,114.6,0.86,1867,2891.64,13.8,5855,4.578,0.797,15.770
2879,1,0,0,0,7.652707,0.970,79.89690,641.1,3.21,989,226.81,254.3,6328,4.481,0.963,13.099
107,0,0,0,0,7.953547,0.300,2.63120,875.4,2.25,696,55.37,38.4,4768,4.536,0.779,15.660
29,0,0,0,0,4.959319,0.831,2.22739,9802.0,12.21,1103,349.40,696.5,5712,4.359,1.082,15.263


# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [9]:


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.utils import to_categorical


In [10]:
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [11]:
# Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

In [12]:
# Convert encoded labels to one-hot-encoding
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

# Train The Model

In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [14]:
# Create model and add layers
model = Sequential()
model.add(Dense(units=50, activation='relu', input_dim=16))
model.add(Dense(units=50, activation='relu'))
model.add(Dense(units=50, activation='relu'))
model.add(Dense(units=3, activation='softmax'))

In [15]:
# Compile and fit the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.summary()

In [16]:
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=100,
    shuffle=True,
    verbose=1
)

Epoch 1/100
164/164 [==============================] - 0s 1ms/step - loss: 0.5765 - accuracy: 0.7047
Epoch 2/100
164/164 [==============================] - 0s 1ms/step - loss: 0.3838 - accuracy: 0.7726
Epoch 3/100
164/164 [==============================] - 0s 1ms/step - loss: 0.3781 - accuracy: 0.7753
Epoch 4/100
164/164 [==============================] - 0s 1ms/step - loss: 0.3752 - accuracy: 0.7786
Epoch 5/100
164/164 [==============================] - 0s 1ms/step - loss: 0.3709 - accuracy: 0.7887
Epoch 6/100
164/164 [==============================] - 0s 1ms/step - loss: 0.3678 - accuracy: 0.7900
Epoch 7/100
164/164 [==============================] - 0s 1ms/step - loss: 0.3650 - accuracy: 0.7910
Epoch 8/100
164/164 [==============================] - 0s 1ms/step - loss: 0.3623 - accuracy: 0.7973
Epoch 9/100
164/164 [==============================] - 0s 1ms/step - loss: 0.3581 - accuracy: 0.8032
Epoch 10/100
164/164 [==============================] - 0s 1ms/step - loss: 0.3574 - accura

164/164 [==============================] - 0s 1ms/step - loss: 0.2592 - accuracy: 0.8817
Epoch 82/100
164/164 [==============================] - 0s 1ms/step - loss: 0.2669 - accuracy: 0.8793
Epoch 83/100
164/164 [==============================] - 0s 1ms/step - loss: 0.2644 - accuracy: 0.8777
Epoch 84/100
164/164 [==============================] - 0s 1ms/step - loss: 0.2532 - accuracy: 0.8842
Epoch 85/100
164/164 [==============================] - 0s 1ms/step - loss: 0.2571 - accuracy: 0.8850
Epoch 86/100
164/164 [==============================] - 0s 1ms/step - loss: 0.2658 - accuracy: 0.8730
Epoch 87/100
164/164 [==============================] - 0s 1ms/step - loss: 0.2517 - accuracy: 0.8856
Epoch 88/100
164/164 [==============================] - 0s 1ms/step - loss: 0.2570 - accuracy: 0.8835
Epoch 89/100
164/164 [==============================] - 0s 1ms/step - loss: 0.2543 - accuracy: 0.8856
Epoch 90/100
164/164 [==============================] - 0s 1ms/step - loss: 0.2524 - accuracy: 

In [17]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

55/55 - 0s - loss: 0.2659 - accuracy: 0.8924
Normal Neural Network - Loss: 0.2658883333206177, Accuracy: 0.8924484848976135


# Make predictions

In [18]:
encoded_predictions = model.predict_classes(X_test_scaled[:5])
prediction_labels = label_encoder.inverse_transform(encoded_predictions)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [19]:

print(f"Predicted classes: {prediction_labels}")
print(f"Actual Labels: {list(y_test[:5])}")

Predicted classes: ['FALSE POSITIVE' 'CANDIDATE' 'FALSE POSITIVE' 'FALSE POSITIVE'
 'FALSE POSITIVE']
Actual Labels: ['FALSE POSITIVE', 'CANDIDATE', 'FALSE POSITIVE', 'FALSE POSITIVE', 'FALSE POSITIVE']


# Save the Model

In [25]:
# save model into new folder

import joblib
filename = 'DeepLearning'

model.save(filename)

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: DeepLearning\assets


In [30]:
# from tensorflow import keras

# a = keras.models.load_model('DeepLearning')

